<h1 style = 'text-align : center'>Big Data And Data Mining Assignment</h1>

<h2 style = 'text-align : center'>Import Necessary Libraries</h2>

In [ ]:
pip install openpyxl

In [ ]:
pip install ydata-profiling --user

In [ ]:
pip install mlxtend

In [ ]:
pip install scikit-learn_extra

In [ ]:
pip install folium

In [ ]:
pip install imbalanced-learn

In [ ]:
pip install xgboost

In [ ]:
import warnings
import sqlite3
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

from pandas_profiling import ProfileReport
import datetime
from datetime import time
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
from sklearn.cluster import DBSCAN
import folium
from scipy import stats
from sklearn.ensemble import IsolationForest
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from numpy import mean
from numpy import std
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, StackingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

%matplotlib inline

In [ ]:
warnings.filterwarnings('ignore')

<h2 style = 'text-align : center'>Create Database Connection and Cursor</h2>

### I created a class Database that does the following:
+ Creates a database connection
+ Creates a database cursor
+ Gets the number of the tables in the database
+ Gets the names of the tables in the database   
+ Retrieves a list of columns of a table in the database
+ Extracts the data from a table in the database and puts it in a dataframe
+ Extracts the data for year 2020 from a table in the database and puts it in a dataframe
+ Closes the database connection

In [ ]:
class Database:

    # Initializing the class Database
    def __init__(self, database):
        self.database = database

        # Creates a connection to the database
        self.con = sqlite3.connect(database)

        # Creates a database cursor
        self.cur = self.con.cursor()

    # Method to get the number of tables in the database
    def get_number_of_tables(self):

        # Gets the number of the tables in the database
        res = self.cur.execute("SELECT COUNT (*) FROM sqlite_master WHERE type = 'table'")

        # Returns the number of tables in the database
        return res.fetchone()

    # Method to get the names of the tables in the database
    def get_table_names(self):

        # Gets the names of the tables in the database
        res = self.cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")

        # Returns the names of the tables in the databbase
        return res.fetchall()

    # Method to get the list of columns in a table in the database
    def get_columns(self, table_name):

        # Query to select the columns from table_name
        self.cur.execute(f"SELECT * FROM {table_name}")

        # Gets the list of columns from cursor description using list comprehension
        columns = [column[0] for column in self.cur.description]

        # Returns a list of columns in a table
        return columns

    # Method to get a dataframe derived from the data of a table in the database
    def dataframe(self, table_name):

        # Selects the data from a table in the database
        query = f"SELECT * FROM {table_name}"

        # Reads the data from the table into a dataframe with pandas
        df = pd.read_sql_query(query, self.con)

        # Returns a dataframe
        return df

    # Method to get a dataframe derived from the data of a table in the database from year 2020 only
    def dataframe_2020(self, table_name):

        # Select the data from 2020 from the specified table
        query = f"SELECT * FROM {table_name} WHERE accident_year = 2020"

        # Read the data from the table into a dataframe with pandas
        df = pd.read_sql_query(query, self.con)

        # Returns a dataframe with data from year 2020 only
        return df

    # Method to close the database connection
    def close_connection(self):

        # Closes the database connection
        self.con.close()

In [ ]:
# Creates an instance of the class Database
# Creates a connection to accident_data_v1.0.0_2023 database and a database cursor

db = Database('accident_data_v1.0.0_2023.db')

In [ ]:
# Gets the number of the tables in the accident_data_v1.0.0_2023 database

db.get_number_of_tables()

In [ ]:
# Gets the names of the tables in the accident_data_v1.0.0_2023 database

db.get_table_names()

In [ ]:
# Gets the list of columns in the accident table

accident_columns = db.get_columns('accident')
accident_columns

In [ ]:
# Gets the list of columns in the casualty table

casualty_columns = db.get_columns('casualty')
casualty_columns

In [ ]:
# Gets the list of columns in the vehicle table

vehicle_columns = db.get_columns('vehicle')
vehicle_columns

In [ ]:
# Gets the list of columns in the lsoa table

lsoa_columns = db.get_columns('lsoa')
lsoa_columns

<h2 style = 'text-align : center'>Presentation Questions</h2>

<h4 style = 'text-align : center'>3A. The age of the oldest driver/rider in the casualty table</h4>

In [ ]:
# Ges the age of the oldest driver/rider in the casualty table

res = db.cur.execute('SELECT max(age_of_casualty) FROM casualty WHERE casualty_class = 1')
res.fetchone()

<h4 style = 'text-align : center'>3B. The total number of vehicle_type = 19 vehicles in the vehicle table</h4>  

In [ ]:
# Gets the total number of vehicle_type = 19 vehicles in the vehicle table

res = db.cur.execute('SELECT COUNT (*) FROM vehicle WHERE vehicle_type = 19')
res.fetchone()

<h4 style = 'text-align : center'>3C. The sex of driver, sex of casualty, speed limit and age of vehicle for accidents in all the lower layer super output area (LSOA) regions of Kingston Upon Hull</h4>  

In [ ]:
# Gets The sex of driver, sex of casualty, speed limit and age of vehicle
# For accidents in all the lower layer super output area (LSOA) regions of Kingston Upon Hull

query = """

                       SELECT sex_of_driver, age_of_vehicle, sex_of_casualty, speed_limit
                       FROM lsoa
                       JOIN accident ON lsoa.lsoa01cd = accident.lsoa_of_accident_location
                       JOIN vehicle ON accident.accident_index = vehicle.accident_index
                       JOIN casualty ON accident.accident_index = casualty.accident_index
                       WHERE lsoa01nm LIKE '%Kingston upon Hull%'

"""

In [ ]:
# Read in a dataframe with the data for the sex of driver, sex of casualty, speed limit and age of vehicle
# For accidents in all the lower layer super output area (LSOA) regions of Kingston Upon Hull

df = pd.read_sql_query(query, db.con)
df

<h1 style = 'text-align : center'>Data Cleaning</h1>

+ For this report I will be working with data for 2020 only
+ I will clean the data with the help of the catalogue of how data was filled in the accident_data_v1.0.0_2023.db

In [ ]:
# Reads in the catalogue of all the columns information in accident_data_v1.0.0_2023 database with pandas

catalogue_df = pd.read_excel('Road-Safety-Open-Dataset-Data-Guide.xlsx')
catalogue_df

I will extract the data from each table for the year 2020 only and read it into a dataframe

In [ ]:
# Dataframe of accident containing data from 2020 only

accident_df = db.dataframe_2020('accident')
accident_df.head()

In [ ]:
# Dataframe of casualty containing data from 2020 only

casualty_df = db.dataframe_2020('casualty')
casualty_df.head()

In [ ]:
# Dataframe of vehicle containing data from 2020 only

vehicle_df = db.dataframe_2020('vehicle')
vehicle_df.head()

This column does not have accident year so I will extract all the data from lsoa table and read it into a dataframe

In [ ]:
# Dataframe of lsoa

lsoa_df = db.dataframe('lsoa')
lsoa_df.head()

In [ ]:
# Closes the connection to the accident_data_v1.0.0_2023 database

db.close_connection()

### I created a class data_cleaning that does the following:

+ Fills in nan values with forward fill
+ Plots the distribution of a column with a histogram
+ Fills in nan values with kde

In [ ]:
class data_cleaning:
    def __init__(self, accident_df, casualty_df, vehicle_df):
        self.accident_df = accident_df
        self.casualty_df = casualty_df
        self.vehicle_df = vehicle_df

    # Method to fill nan values with forward fill
    def use_forward_fill(self, df, columns):

        # Iterating through each column in the list columns
        for column in columns:

            # Fills in each column with nan values with forward fill
            df[column].fillna(method = 'ffill', inplace = True)

    # Method to plot the distribution of a column
    def plot_columns(self, df, column):

        # Plots the distribution of a column with a histgram
        sns.histplot(data = df, x = column)
        plt.title(f'Distribution of {column}')

        # Returns a histogram
        return plt.show()

    # Method to fill in values with kde
    def using_kde(self, df, column):

        # Creates a duplicated version of the dataframe
        duplicated_df = df.copy()

        # Replaces -1 values with nan values
        duplicated_df[column] = duplicated_df[column].replace(-1, np.nan)

        # Assigns the nan values of a column to a variable
        nan_values = duplicated_df[column].isnull()

        # Drops the nan values of a column
        non_nan_values = duplicated_df[column].dropna()

        # Creates an instance of the Gaussian Kde model
        gaussian_kde_model = gaussian_kde(non_nan_values)

        # Gets the sum of the nan values
        num_of_nan_values = np.sum(nan_values)

        # Changes all the nan of the column using kde
        duplicated_df[column].loc[nan_values] = gaussian_kde_model.resample(num_of_nan_values)[0]

        # Changes all the values of the column to integers
        duplicated_df[column] = duplicated_df[column].apply(int)

        df[column] = duplicated_df[column]

In [ ]:
# Creates an instance of the data_cleaning class

clean_data = data_cleaning(accident_df, casualty_df, vehicle_df)

<h3 style = 'text-align : center'>Accident Table</h3>

In [ ]:
# prints accident_df information

accident_df.info()

In [ ]:
# Checks for nan values in accident_df

accident_df.isnull().sum()

In [ ]:
# Check for duplicated columns in accident_df

accident_df.duplicated().sum()

<h4 style = 'text-align : center'>NaN values</h4>

In [ ]:
# Dataframe of accident_df when location_easting_osgr column has nan values

accident_df[accident_df['location_easting_osgr'].isnull()]

+ I noticed there are nan values in the location_easting_osgr, location_northing_osgr, longitude and latitude columns
+ I noticed that all the columns with nan values are in the same rows
+ I filled in the nan values for these columns with forward fill

In [ ]:
# List of columns to fill with forward fill

forward_fill_columns = ['location_easting_osgr','location_northing_osgr', 'longitude', 'latitude']

In [ ]:
# Fills the data in the list columns with forward fill

clean_data.use_forward_fill(accident_df, forward_fill_columns)

<h4 style = 'text-align : center'>speed_limit</h4>

In [ ]:
# Checks catalogue for speed_limit

catalogue_df[(catalogue_df['table'] == 'accident') | (catalogue_df['field name'] == 'speed_limit')]

In [ ]:
# Prints value counts for speed limit

accident_df['speed_limit'].value_counts()

+ I noticed there few -1 values (12) of speed limit
+ I noticed there is a label 99 for unknown(self reported)
+ I filled the -1 values of speed limit with 99

In [ ]:
# Replaces -1 values of speed limit with 99

accident_df['speed_limit'] = accident_df['speed_limit'].replace(-1, 99)

<h4 style = 'text-align : center'>junction_detail</h4>

In [ ]:
# Checks catalogue for junction_detail

catalogue_df[(catalogue_df['table'] == 'accident') | (catalogue_df['field name'] == 'junction_detail')]

In [ ]:
# Prints value counts for junction_detail

accident_df['junction_detail'].value_counts()

+ I noticed there few -1 values (2) of junction_detail
+ I noticed there is a label 99 for unknown(self reported)
+ I filled the -1 values of junction_detail with 99

In [ ]:
# Replaces -1 values of junction_detail with 99

accident_df['junction_detail'] = accident_df['junction_detail'].replace(-1, 99)

<h4 style = 'text-align : center'>junction_control</h4>

In [ ]:
# Checks catalogue for junction_control

catalogue_df[(catalogue_df['table'] == 'accident') | (catalogue_df['field name'] == 'junction_control')]

In [ ]:
# Prints value counts for junction_control

accident_df['junction_control'].value_counts()

+ I noticed there a lot of -1 values (38298) of junction_control
+ I noticed there is a label 9 for unknown(self reported)
+ I filled the -1 values of junction_control with 9

In [ ]:
# Replaces -1 values of junction_control with 9

accident_df['junction_control'] = accident_df['junction_control'].replace(-1, 9)

<h4 style = 'text-align : center'>second_road_class</h4>

In [ ]:
# Checks catalogue for second_road_class

catalogue_df[(catalogue_df['table'] == 'accident') | (catalogue_df['field name'] == 'second_road_class')]

In [ ]:
# Prints value counts for second_road_class

accident_df['second_road_class'].value_counts()

+ I noticed there a lot of -1 values (110) of second_road_class
+ I noticed there is a label 6 for unclassified
+ I filled the -1 values of second_road_class with 6

In [ ]:
# Replaces -1 values of second_road_class with 6

accident_df['second_road_class'] = accident_df['second_road_class'].replace(-1, 6)

<h4 style = 'text-align : center'>second_road_number</h4>

In [ ]:
# Checks catalogue for second_road_number

catalogue_df[(catalogue_df['table'] == 'accident') | (catalogue_df['field name'] == 'second_road_number')]

In [ ]:
# Prints the value counts for second_road_number

accident_df['second_road_number'].value_counts()

+ I noticed there a lot of -1 values (38045) of second_road_number
+ I noticed there is a label 6 for unclassified
+ I filled the -1 values of second_road_number with 0

In [ ]:
# Replaces -1 values of second_road_number with 0

accident_df['second_road_number'] = accident_df['second_road_number'].replace(-1, 0)

<h4 style = 'text-align : center'>pedestrian_crossing_human_control</h4>

In [ ]:
# Checks catalogue for pedestrian_crossing_human_control

catalogue_df[(catalogue_df['table'] == 'accident') | (catalogue_df['field name'] == 'pedestrian_crossing_human_control')]

In [ ]:
# Prints the value counts for pedestrian_crossing_human_control

accident_df['pedestrian_crossing_human_control'].value_counts()

+ I noticed there a lot of -1 values (143) of pedestrian_crossing_human_control
+ I noticed there is a label 9 for unknown(self reported)
+ I filled the -1 values of pedestrian_crossing_human_control with 9

In [ ]:
# Replaces -1 values of second_road_number with 0

accident_df['second_road_number'] = accident_df['second_road_number'].replace(-1, 0)

<h4 style = 'text-align : center'>pedestrian_crossing_physical_facilities</h4>

In [ ]:
# Checks catalogue for pedestrian_crossing_physical_facilities

catalogue_df[(catalogue_df['table'] == 'accident') | (catalogue_df['field name'] == 'pedestrian_crossing_physical_facilities')]

In [ ]:
# Prints the value counts for pedestrian_crossing_physical_facilities

accident_df['pedestrian_crossing_physical_facilities'].value_counts()

+ I noticed there a lot of -1 values (135) of pedestrian_crossing_physical_facilities
+ I noticed there is a label 9 for unknown(self reported)
+ I filled the -1 values of pedestrian_crossing_physical_facilities with 9

In [ ]:
# Replaces -1 values of pedestrian_crossing_physical_facilities with 9

accident_df['pedestrian_crossing_physical_facilities'] = accident_df['pedestrian_crossing_physical_facilities'].replace(-1, 9)

<h4 style = 'text-align : center'>light_conditions</h4>

In [ ]:
# Checks catalogue for light_conditions

catalogue_df[(catalogue_df['table'] == 'accident') | (catalogue_df['field name'] == 'light_conditions')]

In [ ]:
# Prints the value counts for light_conditions

accident_df['light_conditions'].value_counts()

+ I noticed there a few -1 values (1) of light_conditions
+ I noticed there is a label 7 for darkness - lighting unknown
+ I filled the -1 values of light_conditions with 7

In [ ]:
# Replaces -1 values of light_conditions with 7

accident_df['light_conditions'] = accident_df['light_conditions'].replace(-1, 7)

<h4 style = 'text-align : center'>weather_conditions</h4>

In [ ]:
# Checks catalogue for weather_conditions

catalogue_df[(catalogue_df['table'] == 'accident') | (catalogue_df['field name'] == 'weather_conditions')]

In [ ]:
# Prints the value counts for weather_conditions

accident_df['weather_conditions'].value_counts()

+ I noticed there a few -1 values (1) of weather_conditions
+ I noticed there is a label 9 for Unknown
+ I filled the -1 values of weather_conditions with 9

In [ ]:
# Replaces -1 values of weather_conditions with 9

accident_df['weather_conditions'] = accident_df['weather_conditions'].replace(-1, 9)

<h4 style = 'text-align : center'>road_surface_conditions</h4>

In [ ]:
# Checks catalogue for road_surface_conditions

catalogue_df[(catalogue_df['table'] == 'accident') | (catalogue_df['field name'] == 'road_surface_conditions')]

In [ ]:
# Prints the value counts for road_surface_conditions

accident_df['road_surface_conditions'].value_counts()

+ I noticed there a lot of -1 values (316) of road_surface_conditions
+ I noticed there is a label 9 for unknown(self reported)
+ I filled the -1 values of road_surface_conditions with 9

In [ ]:
# Replaces -1 values of road_surface_conditions with 9

accident_df['road_surface_conditions'] = accident_df['road_surface_conditions'].replace(-1, 9)

<h4 style = 'text-align : center'>special_conditions_at_site</h4>

In [ ]:
# Checks catalogue for special_conditions_at_site

catalogue_df[(catalogue_df['table'] == 'accident') | (catalogue_df['field name'] == 'special_conditions_at_site')]

In [ ]:
# Prints the value counts for special_conditions_at_site

accident_df['special_conditions_at_site'].value_counts()

+ I noticed there a lot of -1 values (218) of special_conditions_at_site
+ I noticed there is a label 9 for unknown(self reported)
+ I filled the -1 values of special_conditions_at_site with 9

In [ ]:
# Replaces -1 values of special_conditions_at_site with 9

accident_df['special_conditions_at_site'] = accident_df['special_conditions_at_site'].replace(-1, 9)

<h4 style = 'text-align : center'>carriageway_hazards</h4>

In [ ]:
# Checks catalogue for carriageway_hazards

catalogue_df[(catalogue_df['table'] == 'accident') | (catalogue_df['field name'] == 'carriageway_hazards')]

In [ ]:
# Prints the value counts for carriageway_hazards

accident_df['carriageway_hazards'].value_counts()

+ I noticed there a lot of -1 values (208) of carriageway_hazards
+ I noticed there is a label 9 for unknown(self reported)
+ I filled the -1 values of carriageway_hazards_at_site with 9

In [ ]:
# Replaces -1 values of carriageway_hazards_at_site with 9

accident_df['carriageway_hazards'] = accident_df['carriageway_hazards'].replace(-1, 9)

<h4 style = 'text-align : center'>trunk_road_flag</h4>

In [ ]:
# Checks catalogue for trunk_road_flag

catalogue_df[(catalogue_df['table'] == 'accident') | (catalogue_df['field name'] == 'trunk_road_flag')]

In [ ]:
# Dataframe for -1 values of trunk_road_flag

no_trunk = accident_df[accident_df['trunk_road_flag'] == -1]
no_trunk

In [ ]:
# Unique values of local_authority_ons_district  for the dataframe of -1 values of trunk_road_flag

no_trunk['local_authority_ons_district'].unique()

+ I noticed all the column of -1 values of trunk_road_flag are in Wales or Scotland from the local_authority_ons_district column
+ For a road to be trunk it has to be managed by Highways England
+ I filled in the -1 values of trunk_road_flag with 2 which is for non_trunk

In [ ]:
# Replace trunk_road_flag -1 with 2 (non trunk)

accident_df.loc[accident_df['trunk_road_flag'] == -1, 'trunk_road_flag'] = 2

<h3 style = 'text-align : center'>Casualty Table</h3>

In [ ]:
# prints casualty_df information

casualty_df.info()

In [ ]:
# Check for NaN values in casualty_df

casualty_df.isnull().sum()

In [ ]:
# Check for duplicated values in casualty_df

casualty_df.duplicated().sum()

<h4 style = 'text-align : center'>sex_of_casualty</h4>

In [ ]:
# Checks catalogue for sex_of_casualty

catalogue_df[(catalogue_df['table'] == 'casualty') | (catalogue_df['field name'] == 'sex_of_casualty')]

In [ ]:
# Prints the value counts for sex_of_casualty

casualty_df['sex_of_casualty'].value_counts()

+ I noticed there a lot of -1 values (756) of sex_of_casualty
+ I noticed there is a label 9 for unknown(self reported)
+ I filled the -1 values of sex_of_casualty with 9

In [ ]:
# Replaces -1 values of sex_of_casualty with 9

casualty_df['sex_of_casualty'] = casualty_df['sex_of_casualty'].replace(-1, 9)

<h4 style = 'text-align : center'>age_of_casualty</h4>

In [ ]:
# Distribution of age_of_casualty

clean_data.plot_columns(casualty_df, 'age_of_casualty')

In [ ]:
# Fills in values of age_of_casualty with kde

clean_data.using_kde(casualty_df, 'age_of_casualty')

In [ ]:
# Distribution of age_of_casualty after cleaning with kde

clean_data.plot_columns(casualty_df, 'age_of_casualty')

<h4 style = 'text-align : center'>pedestrian_location</h4>

In [ ]:
# Checks catalogue for pedestrian_location

catalogue_df[(catalogue_df['table'] == 'casualty') | (catalogue_df['field name'] == 'pedestrian_location')]

In [ ]:
# Prints the value counts for pedestrian_location

casualty_df['pedestrian_location'].value_counts()

+ I noticed there a few -1 values (2) of pedestrian_location
+ I noticed there is a label 10 for unknown or other
+ I filled the -1 values of pedestrian_location with 10

In [ ]:
# Replaces -1 values of pedestrian_location with 10

casualty_df['pedestrian_location'] = casualty_df['pedestrian_location'].replace(-1, 10)

<h4 style = 'text-align : center'>pedestrian_movement</h4>

In [ ]:
# Checks catalogue for pedestrian_movement

catalogue_df[(catalogue_df['table'] == 'casualty') | (catalogue_df['field name'] == 'pedestrian_movement')]

In [ ]:
# Prints the value counts for pedestrian_movement

casualty_df['pedestrian_movement'].value_counts()

+ I noticed there a few -1 values (2) of pedestrian_movement
+ I noticed there is a label 9 for unknown or other
+ I filled the -1 values of pedestrian_movement with 9

In [ ]:
# Replaces -1 values of pedestrian_movement with 9

casualty_df['pedestrian_movement'] = casualty_df['pedestrian_movement'].replace(-1, 9)

<h4 style = 'text-align : center'>car_passenger</h4>

In [ ]:
# Checks catalogue for car_passenger

catalogue_df[(catalogue_df['table'] == 'casualty') | (catalogue_df['field name'] == 'car_passenger')]

In [ ]:
# Prints the value counts for car_passenger

casualty_df['car_passenger'].value_counts()

+ I noticed there a lot of -1 values (311) of car_passenger
+ I noticed there is a label 9 for unknown(self reported)
+ I filled the -1 values of car_passenger with 9

In [ ]:
# Replaces -1 values of car_passenger with 9

casualty_df['car_passenger'] = casualty_df['car_passenger'].replace(-1, 9)

<h4 style = 'text-align : center'>bus_or_coach_passenger</h4>

In [ ]:
# Checks catalogue for bus_or_coach_passenger

catalogue_df[(catalogue_df['table'] == 'casualty') | (catalogue_df['field name'] == 'bus_or_coach_passenger')]

In [ ]:
# Prints the value counts for bus_or_coach_passenger

casualty_df['bus_or_coach_passenger'].value_counts()

+ I noticed there few -1 values (22) of bus_or_coach_passenger
+ I noticed there is a label 9 for unknown(self reported)
+ I filled the -1 bus_or_coach_passenger of sex_of_casualty with 9

In [ ]:
# Replaces -1 values of bus_or_coach_passenger with 9

casualty_df['bus_or_coach_passenger'] = casualty_df['bus_or_coach_passenger'].replace(-1, 9)

<h4 style = 'text-align : center'>pedestrian_road_maintenance_worker</h4>

In [ ]:
# Checks catalogue for pedestrian_road_maintenance_worker

catalogue_df[(catalogue_df['table'] == 'casualty') | (catalogue_df['field name'] == 'pedestrian_road_maintenance_worker')]

In [ ]:
# Prints the value counts for pedestrian_road_maintenance_worker

casualty_df['pedestrian_road_maintenance_worker'].value_counts()

+ I noticed there a lot of -1 values (94) of pedestrian_road_maintenance_worker
+ I noticed there is a label 2 for not known
+ I filled the -1 values of pedestrian_road_maintenance_worker with 2

In [ ]:
# Replaces -1 values of pedestrian_road_maintenance_worker with 2

casualty_df['pedestrian_road_maintenance_worker'] = casualty_df['pedestrian_road_maintenance_worker'].replace(-1, 2)

<h3 style = 'text-align : center'>Vehicle Table</h3>

In [ ]:
# Print vehicle_df information

vehicle_df.info()

In [ ]:
# Check for NaN values in vehicle_df

vehicle_df.isnull().sum()

In [ ]:
# Check for duplicated values in vehicle_df

vehicle_df.duplicated().sum()

<h4 style = 'text-align : center'>towing_and_articulation</h4>

In [ ]:
# Checks catalogue for towing_and_articulation

catalogue_df[(catalogue_df['table'] == 'vehicle') | (catalogue_df['field name'] == 'towing_and_articulation')]

In [ ]:
# Prints the value counts for towing_and_articulation

vehicle_df['towing_and_articulation'].value_counts()

+ I noticed there a lot -1 values (689) of towing_and_articulation
+ I noticed there is a label 9 for unknown(self reported)
+ I filled the -1 of towing_and_articulation with 9

In [ ]:
# Replaces -1 values of towing_and_articulation with 2

vehicle_df['towing_and_articulation'] = vehicle_df['towing_and_articulation'].replace(-1, 2)

<h4 style = 'text-align : center'>vehicle_manoeuvre</h4>

In [ ]:
# Checks catalogue for vehicle_manoeuvre

catalogue_df[(catalogue_df['table'] == 'vehicle') | (catalogue_df['field name'] == 'vehicle_manoeuvre')]

In [ ]:
# Prints the value counts for vehicle_manoeuvre

vehicle_df['vehicle_manoeuvre'].value_counts()

+ I noticed there a lot of -1 values (679) of vehicle_manoeuvre
+ I noticed there is a label 99 for unknown(self reported)
+ I filled the -1 values of vehicle_manoeuvre with 99

In [ ]:
# Replaces -1 values of vehicle_manoeuvre with 99

vehicle_df['vehicle_manoeuvre'] = vehicle_df['vehicle_manoeuvre'].replace(-1, 99)

<h4 style = 'text-align : center'>vehicle_direction_from</h4>

In [ ]:
# Checks catalogue for vehicle_direction_from

catalogue_df[(catalogue_df['table'] == 'vehicle') | (catalogue_df['field name'] == 'vehicle_direction_from')]

In [ ]:
# Prints the value counts for vehicle_direction_from

vehicle_df['vehicle_direction_from'].value_counts()

+ I noticed there a lot of -1 values (1640) of vehicle_direction_from
+ I noticed there is a label 9 for unknown(self reported)
+ I filled the -1 vehicle_direction_from of sex_of_casualty with 9

In [ ]:
# Replaces -1 values of vehicle_direction_from with 9

vehicle_df['vehicle_direction_from'] = vehicle_df['vehicle_direction_from'].replace(-1, 9)

<h4 style = 'text-align : center'>vehicle_direction_to</h4>

In [ ]:
# Checks catalogue for vehicle_direction_to

catalogue_df[(catalogue_df['table'] == 'vehicle') | (catalogue_df['field name'] == 'vehicle_direction_to')]

In [ ]:
# Prints the value counts for vehicle_direction_to

vehicle_df['vehicle_direction_to'].value_counts()

+ I noticed there a lot of -1 values (1653) of vehicle_direction_to
+ I noticed there is a label 9 for unknown(self reported)
+ I filled the -1 values of vehicle_direction_to with 9

In [ ]:
# Replaces -1 values of vehicle_direction_to with 9

vehicle_df['vehicle_direction_to'] = vehicle_df['vehicle_direction_to'].replace(-1, 9)

<h4 style = 'text-align : center'>vehicle_location_restricted_lane</h4>

In [ ]:
# Checks catalogue for vehicle_location_restricted_lane

catalogue_df[(catalogue_df['table'] == 'vehicle') | (catalogue_df['field name'] == 'vehicle_location_restricted_lane')]

In [ ]:
# Prints the value counts for vehicle_location_restricted_lane

vehicle_df['vehicle_location_restricted_lane'].value_counts()

+ I noticed there a lot of -1 values (640) of vehicle_location_restricted_lane
+ I noticed there is a label 99 for unknown(self reported)
+ I filled the -1  of vehicle_location_restricted_lane with 99

In [ ]:
# Replaces -1 values of vehicle_location_restricted_lane with 99

vehicle_df['vehicle_location_restricted_lane'] = vehicle_df['vehicle_location_restricted_lane'].replace(-1, 99)

<h4 style = 'text-align : center'>junction_location</h4>

In [ ]:
# Checks catalogue for junction_location

catalogue_df[(catalogue_df['table'] == 'vehicle') | (catalogue_df['field name'] == 'junction_location')]

In [ ]:
# Prints the value counts for junction_location

vehicle_df['junction_location'].value_counts()

+ I noticed there a lot of -1 values (284) of junction_location
+ I noticed there is a label 9 for unknown(self reported)
+ I filled the -1 values of junction_location with 9

In [ ]:
# Replaces -1 values of junction_location with 9

vehicle_df['junction_location'] = vehicle_df['junction_location'].replace(-1, 9)

<h4 style = 'text-align : center'>skidding_and_overturning</h4>

In [ ]:
# Checks catalogue for skidding_and_overturning

catalogue_df[(catalogue_df['table'] == 'vehicle') | (catalogue_df['field name'] == 'skidding_and_overturning')]

In [ ]:
# Prints the value counts for skidding_and_overturning

vehicle_df['skidding_and_overturning'].value_counts()

+ I noticed there a lot of -1 values (662) of skidding_and_overturning
+ I noticed there is a label 9 for unknown(self reported)
+ I filled the -1 values of skidding_and_overturning with 9

In [ ]:
# Replaces -1 values of skidding_and_overturning with 9

vehicle_df['skidding_and_overturning'] = vehicle_df['skidding_and_overturning'].replace(-1, 9)

<h4 style = 'text-align : center'>hit_object_in_carriageway</h4>

In [ ]:
# Checks catalogue for hit_object_in_carriageway

catalogue_df[(catalogue_df['table'] == 'vehicle') | (catalogue_df['field name'] == 'hit_object_in_carriageway')]

In [ ]:
# Prints the value counts for hit_object_in_carriageway

vehicle_df['hit_object_in_carriageway'].value_counts()

+ I noticed there a lot of -1 values (636) of hit_object_in_carriageway
+ I noticed there is a label 99 for unknown(self reported)
+ I filled the -1 values of hit_object_in_carriageway with 99

In [ ]:
# Replaces -1 values of hit_object_in_carriageway with 99

vehicle_df['hit_object_in_carriageway'] = vehicle_df['hit_object_in_carriageway'].replace(-1, 99)

<h4 style = 'text-align : center'>vehicle_leaving_carriageway</h4>

In [ ]:
# Checks catalogue for vehicle_leaving_carriageway

catalogue_df[(catalogue_df['table'] == 'vehicle') | (catalogue_df['field name'] == 'vehicle_leaving_carriageway')]

In [ ]:
# Prints the value counts for vehicle_leaving_carriageway

vehicle_df['vehicle_leaving_carriageway'].value_counts()

+ I noticed there a lot of -1 values (644) of vehicle_leaving_carriageway
+ I noticed there is a label 9 for unknown(self reported)
+ I filled the -1 values of vehicle_leaving_carriageway with 9

In [ ]:
# Replaces -1 values of vehicle_leaving_carriageway

vehicle_df['vehicle_leaving_carriageway'] = vehicle_df['vehicle_leaving_carriageway'].replace(-1, 9)

<h4 style = 'text-align : center'>hit_object_off_carriageway</h4>

In [ ]:
# Checks catalogue for hit_object_off_carriageway

catalogue_df[(catalogue_df['table'] == 'vehicle') | (catalogue_df['field name'] == 'hit_object_off_carriageway')]

In [ ]:
# Prints the value counts for hit_object_off_carriageway

vehicle_df['hit_object_off_carriageway'].value_counts()

+ I noticed there a few -1 values (4) of hit_object_off_carriageway
+ I noticed there is a label 99 for unknown(self reported)
+ I filled the -1 values of hit_object_off_carriageway with 99

In [ ]:
# Replaces -1 values of hit_object_off_carriageway with 99

vehicle_df['hit_object_off_carriageway'] = vehicle_df['hit_object_off_carriageway'].replace(-1, 99)

<h4 style = 'text-align : center'>first_point_of_impact</h4>

In [ ]:
# Checks catalogue for first_point_of_impact

catalogue_df[(catalogue_df['table'] == 'vehicle') | (catalogue_df['field name'] == 'first_point_of_impact')]

In [ ]:
# Prints the value counts for first_point_of_impact

vehicle_df['first_point_of_impact'].value_counts()

+ I noticed there a lot of -1 values (939) of first_point_of_impact
+ I noticed there is a label 9 for unknown(self reported)
+ I filled the -1 values of first_point_of_impact with 9

In [ ]:
# Replaces -1 values of first_point_of_impact with 9

vehicle_df['first_point_of_impact'] = vehicle_df['first_point_of_impact'].replace(-1, 9)

<h4 style = 'text-align : center'>vehicle_left_hand_drive</h4>

In [ ]:
# Checks catalogue for vehicle_left_hand_drive

catalogue_df[(catalogue_df['table'] == 'vehicle') | (catalogue_df['field name'] == 'vehicle_left_hand_drive')]

In [ ]:
# Prints the value counts for vehicle_left_hand_drive

vehicle_df['vehicle_left_hand_drive'].value_counts()

+ I noticed there a lot of -1 values (877) of vehicle_left_hand_drive
+ I noticed there is a label 9 for unknown(self reported)
+ I filled the -1 values of vehicle_left_hand_drive with 9

In [ ]:
# Replaces -1 values of vehicle_left_hand_drive

vehicle_df['vehicle_left_hand_drive'] = vehicle_df['vehicle_left_hand_drive'].replace(-1, 9)

<h4 style = 'text-align : center'>journey_purpose_of_driver</h4>

In [ ]:
# Checks catalogue for journey_purpose_of_driver

catalogue_df[(catalogue_df['table'] == 'vehicle') | (catalogue_df['field name'] == 'journey_purpose_of_driver')]

In [ ]:
# Prints the value counts for journey_purpose_of_driver

vehicle_df['journey_purpose_of_driver'].value_counts()

+ I noticed there a lot of -1 values (184) of journey_purpose_of_driver
+ I noticed there is a label 15 for unknown(self reported)
+ I filled the -1 values of journey_purpose_of_driver with 15

In [ ]:
# Replaces -1 values of journey_purpose_of_driver with 15

vehicle_df['journey_purpose_of_driver'] = vehicle_df['journey_purpose_of_driver'].replace(-1, 15)

<h4 style = 'text-align : center'>sex_of_driver</h4>

In [ ]:
# Checks catalogue for sex_of_driver

catalogue_df[(catalogue_df['table'] == 'vehicle') | (catalogue_df['field name'] == 'sex_of_driver')]

In [ ]:
# Prints the value counts for sex_of_driver

vehicle_df['sex_of_driver'].value_counts()

+ I noticed there a few -1 values (13) of sex_of_driver
+ I noticed there is a label 9 for unknown(self reported)
+ I filled the -1 values of sex_of_driver with 3

In [ ]:
# Replaces -1 values of sex_of_driver

vehicle_df['sex_of_driver'] = vehicle_df['sex_of_driver'].replace(-1, 3)

<h4 style = 'text-align : center'>age_of_driver</h4>

In [ ]:
# Distribution of age_of_driver

clean_data.plot_columns(vehicle_df, 'age_of_driver')

In [ ]:
# Fills in values of age_of_driver with kde

clean_data.using_kde(vehicle_df, 'age_of_driver')

In [ ]:
# Distribution of age_of_driver after cleaning with kde


clean_data.plot_columns(vehicle_df, 'age_of_driver')

<h4 style = 'text-align : center'>engine_capacity_cc</h4>

In [ ]:
# Checks catalogue for engine_capacity_cc

catalogue_df[(catalogue_df['table'] == 'vehicle') | (catalogue_df['field name'] == 'engine_capacity_cc')]

In [ ]:
# Checks catalogue for vehicle_type

catalogue_df[(catalogue_df['table'] == 'vehicle') | (catalogue_df['field name'] == 'vehicle_type')]

In [ ]:
# Prints the value counts for engine_capacity_cc

vehicle_df['engine_capacity_cc'].value_counts()

#### I noticed there a few -1 values (43603) of engine_capacity_cc
+ I noticed there are vehicle types like horse, pedal cycle that don't have engines so i replaced it with 0
+ I filled the 0 values with kde
+ I noticed there is a label 99 for unknown(self rep only)
+ I filled the rest of the values with 99

In [ ]:
# Replaces -1 values of engine capacity where we have a horse, pedal cycle, etc with 0

vehicle_df.loc[((vehicle_df["engine_capacity_cc"] == -1) &
                ((vehicle_df["vehicle_type"] == 1) |
                 (vehicle_df["vehicle_type"] == 18)|
                 (vehicle_df["vehicle_type"] == 16)|
                 (vehicle_df["vehicle_type"] == 22)|
                 (vehicle_df["vehicle_type"] == 23)
                )
               ), "engine_capacity_cc"] = 0

In [ ]:
# Distribution of age_of_vehicle

clean_data.plot_columns(vehicle_df, 'engine_capacity_cc')

In [ ]:
# Fills in values of age_of_vehicle with kde

clean_data.using_kde(vehicle_df, 'engine_capacity_cc')

In [ ]:
# Distribution of age_of_vehicle after cleaning with kde

clean_data.plot_columns(vehicle_df, 'engine_capacity_cc')

In [ ]:
# Replaces -1 values of engine_capacity_cc

vehicle_df['engine_capacity_cc'] = vehicle_df['engine_capacity_cc'].replace(-1, 99)

<h4 style = 'text-align : center'>age_of_vehicle</h4>

In [ ]:
# Distribution of age_of_vehicle

clean_data.plot_columns(vehicle_df, 'age_of_vehicle')

In [ ]:
# Fills in values of age_of_vehicle with kde

clean_data.using_kde(vehicle_df, 'age_of_vehicle')

In [ ]:
# Distribution of age_of_vehicle after cleaning with kde

clean_data.plot_columns(vehicle_df, 'age_of_vehicle')

<h3 style = 'text-align : center'>Lsoa Table</h3>

In [ ]:
# Print the information for lsao

lsoa_df.info()

In [ ]:
# Check for NaN values in lsoa

lsoa_df.isnull().sum()

In [ ]:
# Check for duplicated values in lsoa_df

lsoa_df.duplicated().sum()

<h1 style = 'text-align : center'>Data Visualization</h1>

### I created profile reports for accident_df, casualty_df, vehicle_df, lsoa_df

# Create a profile report for accident_df

accident_report = profile_report(database)
accident_report

# Create a profile report for casualty_df

casualty_report = ProfileReport(casualty_df)
casualty_report

# Create a profile report for vehicle_df

vehicle_report = ProfileReport(vehicle_df)
vehicle_report

# Create a profile report for lsoa_df

lsoa_report = ProfileReport(lsoa_df)
lsoa_report

<h4 style = 'text-align : center'>1. Are there significant hours of the day, and days of the week, on which accidents occur?</h4>

In [ ]:
# Check catalogue_df for time

catalogue_df[(catalogue_df['table'] == 'accident') | (catalogue_df['field name'] == 'time')]

In [ ]:
# Check catalogue_df for day_of_week

catalogue_df[(catalogue_df['table'] == 'accident') | (catalogue_df['field name'] == 'day_of_week')]

### I created a class accident_periods that does the following:
+ Gets the decimal time
+ Plots the distrubution of accidents by the hour of the day
+ Plots the distrubution of accidents by the day of the week

In [ ]:
class accident_periods:

    def __init__(self, dataframe):
        self.dataframe = dataframe

    def decimal_time(self):

        # Convert the time column to datetime format
        self.dataframe['converted_time'] = pd.DatetimeIndex(self.dataframe['time'])

        # Calculate the hour from the time column
        hour = self.dataframe['converted_time'].dt.hour

        # Calculate the minute from the time column
        minute = self.dataframe['converted_time'].dt.minute

        # Calculate the decimal time
        self.dataframe['decimal_time'] = round(hour + minute /60, 2)


    def plot_hour_of_day(self, title):

        # plot histplot of accidents by hour of the day
        plot = sns.histplot(self.dataframe['decimal_time'], kde = True, bins = 24, color = 'pink')

        # Loop to get the number of accidents for each hour of the day
        for count in plot.patches:

            counts = count.get_height()

            plt.text(count.get_x() + count.get_width() / 2, counts + 0.5, int(counts), ha ='center', fontsize = 8)

        plt.xlabel('Hour of the Day')
        plt.ylabel('Number of Accidents')
        plt.title(f'Distribution of {title} Accidents by Hour of the Day')

        return plt.show()

    def plot_day_of_week(self, title):

        # plot countplot of accidents by day of the week
        day_plot = sns.countplot(data = self.dataframe, x = 'day_of_week', color = 'pink')

        # List of each day of the week
        days_of_week = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

        # Loop to get the number of accidents for each day of the week
        for count in day_plot.patches:

            height = count.get_height()

            plt.text(count.get_x() + count.get_width() / 2,  height + 0.5, str( height), ha ='center', fontsize = 10)


        plt.xlabel('Day of the Week')
        plt.ylabel('Number of Accidents')
        plt.title(f' Distribution of {title} Accidents by Day of the Week')
        day_plot.set_xticklabels(days_of_week, rotation = 45)

        return plt.show()

#### Hours of the day and days of the week which accidents occur

In [ ]:
# Creates an instance of the accident_periods class

accidents = accident_periods(accident_df)

In [ ]:
# Get decimal_time for accidents

accidents.decimal_time()

In [ ]:
# Plot the distribution of accidents by the hour of the day

accidents.plot_hour_of_day('')

In [ ]:
# plot the distribution of accidents by the day of the week

accidents.plot_day_of_week('')

<h4 style = 'text-align : center'>2. For motorbikes, are there significant hours of the day, and days of the week, on which accidents occur? </h4>

In [ ]:
# Check catalogue_df for vehicle_type

catalogue_df[(catalogue_df['table'] == 'vehicle') | (catalogue_df['field name'] == 'vehicle_type')]

#### Hours of the day and days of the week which accidents of motorcycles 125cc and under occur

In [ ]:
# Dataframe of vehicle type 2 & 3 (motorcycle 125cc and under)

# Filter rows from vehicle_df where vehicle type = 2 & 3
vehicle_type = vehicle_df[(vehicle_df['vehicle_type'] == 2) | (vehicle_df['vehicle_type'] == 3)][["accident_index",
                                                                                                   "vehicle_type"]]

# Select accident_index, time & day_of_week from accident_df
accident_info = accident_df[["accident_index", "time", "day_of_week"]]

# Merge the two dataframes, vehicle_type = 2 & 3 & accident_info on accident_index
Motorcycle_125cc_and_under_df = pd.merge(vehicle_type, accident_info, on = 'accident_index')

Motorcycle_125cc_and_under_df

In [ ]:
# Creates an instance of the accident_periods class for accidents_for_motorcycles_125cc_and_under

accidents_for_motorcycles_125cc_and_under = accident_periods(Motorcycle_125cc_and_under_df)

In [ ]:
# Get the decimal time for motorcycles 125cc and under accidents

accidents_for_motorcycles_125cc_and_under.decimal_time()

In [ ]:
# Plot the distribution of motorcycles 125cc and under accidents by the hour of the day

accidents_for_motorcycles_125cc_and_under.plot_hour_of_day('Motorcycles 125cc and under')

In [ ]:
# Plot the distribution of motorcycles 125cc and under accidents by the day of the week

accidents_for_motorcycles_125cc_and_under.plot_day_of_week('Motorcycles 125cc and Under')

#### Hours of the day and days of the week which accidents of motorcycles over 125cc to 500cc

In [ ]:
# Dataframe of vehicle type 4 (125cc and up to 500cc)

# Filter rows from vehicle_df where vehicle type = 4
vehicle_type = vehicle_df[vehicle_df['vehicle_type'] == 4][['accident_index', 'vehicle_type']]

# Select accident_index, time & day_of_week from accident_df
accident_info = accident_df[["accident_index", "time", "day_of_week"]]

# Merge the two dataframes, vehicle_type_4 & accident_info on accident_index
Motorcycle_over_125cc_to_500cc_df = pd.merge(vehicle_type, accident_info, on = 'accident_index')

Motorcycle_over_125cc_to_500cc_df

In [ ]:
# Creates an instance of the class accident_periods for accidents_for_motorcycles_125cc_and_under

accidents_for_motorcycles_over_125cc_to_500cc = accident_periods(Motorcycle_over_125cc_to_500cc_df)

In [ ]:
# Get the decimal time for motorcycles over 125cc to 500cc accidents

accidents_for_motorcycles_over_125cc_to_500cc.decimal_time()

In [ ]:
# Plot the distribution of motorcycles over 125cc to 500cc accidents by the hour of the day

accidents_for_motorcycles_over_125cc_to_500cc.plot_hour_of_day('Motorcycles over 125cc to 500cc')

In [ ]:
# Plot the distribution of motorcycles over 125cc to 500cc accidents by the hour of the day

accidents_for_motorcycles_over_125cc_to_500cc.plot_day_of_week('Motorcycles over 125cc to 500cc')

#### Hours of the day and days of the week which accidents motorcycle over 500cc

In [ ]:
# Dataframe of vehicle type 5 (Motorcycle over 500cc)

# Filter rows from vehicle_df where vehicle type = 5
vehicle_type = vehicle_df[vehicle_df['vehicle_type'] == 5][['accident_index', 'vehicle_type']]

# Select accident_index, time & day_of_week from accident_df
accident_info = accident_df[["accident_index", "time", "day_of_week"]]

# Merge the two dataframes, vehicle_type_5 & accident_info on accident_index
Motorcycle_over_500cc_df = pd.merge(vehicle_type, accident_info, on = 'accident_index')

Motorcycle_over_500cc_df

In [ ]:
# Creates an instance of the class accident_periods for accidents_for_motorcycles_over_500cc

accidents_for_motorcycles_over_500cc = accident_periods(Motorcycle_over_500cc_df)

In [ ]:
# Get the decimal time for motorcycles over 500cc accidents

accidents_for_motorcycles_over_500cc.decimal_time()

In [ ]:
# Plot the distribution of motorcycles over 500cc accidents by the hour of the day

accidents_for_motorcycles_over_500cc.plot_hour_of_day('Motorcycles over 500cc')

In [ ]:
# Plot the distribution of motorcycles over 500cc accidents by the hour of the day

accidents_for_motorcycles_over_500cc.plot_day_of_week('Motorcycles over 500cc')

<h4 style = 'text-align : center'>3. For pedestrians involved in accidents, are there significant hours of the day, and days of the week, on which they are more likely to be involved</h4>

In [ ]:
# Check catalogue_df for casualty_class

catalogue_df[(catalogue_df['table'] == 'casualty') | (catalogue_df['field name'] == 'casualty_class')]

#### Hours of the day and days of the week which accidents of pedestrians occur

In [ ]:
# Dataframe of casualty_class 3 (Pedestrian)

# Filter rows from casualty_df where casualty_class = 3
casualty_class = casualty_df[casualty_df['casualty_class'] == 3][["accident_index", "casualty_class"]]

# Select accident_index, time & day_of_week from accident_df
accident_info = accident_df[["accident_index", "time", "day_of_week"]]

# Merge the two dataframes, casualty_class_3 & accident_info on accident_index
pedestrian_df = pd.merge(casualty_class, accident_info, on = 'accident_index')

pedestrian_df

In [ ]:
# Creates an instance of the class accident_periods for accidents_for_pedestrians

accidents_for_pedestrians = accident_periods(pedestrian_df)

In [ ]:
# Get the decimal time for pedestrian accidents

accidents_for_pedestrians.decimal_time()

In [ ]:
# Plot the distribution of pedestrian accidents by the hour of the day

accidents_for_pedestrians.plot_hour_of_day('Pedestrian')

In [ ]:
# Plot the distribution of pedestrian accidents by the hour of the day

accidents_for_pedestrians.plot_day_of_week('Pedestrian')

<h4 style = 'text-align : center'>4. Using the apriori algorithm, explore the impact of selected variables on accident severity</h4>

#### I ran apriori on some selected variables on accident severity
+ My selected variables are speed_limit and urban_or_rural_area

<h5 style = 'text-align : center'>Understanding the columns</h5>

In [ ]:
# Check catalogue_df for accident_severity

catalogue_df[(catalogue_df['table'] == 'casualty') | (catalogue_df['field name'] == 'accident_severity')]

In [ ]:
# Check catalogue_df for speed_limit

catalogue_df[(catalogue_df['table'] == 'casualty') | (catalogue_df['field name'] == 'speed_limit')]

In [ ]:
# Check catalogue_df for urban_or_rural_area

catalogue_df[(catalogue_df['table'] == 'casualty') | (catalogue_df['field name'] == 'urban_or_rural_area')]

#### I created a class accident_apriori that does the following:
+ Gets my selected variables that i will run apriori on and puts them in a dataframe
+ One hot encodes the dataframe of my selected variables
+ Runs apriori on my one hot encoded dataframe
+ Generates apriori association rules  
+ Filters association rules relevant to accident_severity

In [ ]:
class accident_apriori:

    def __init__(self, dataframe):
        self.dataframe = dataframe

    def get_dataframe_of_selected_variables(self, columns_to_encode):

        df = self.dataframe[columns_to_encode]

        return df

    def one_hot_encode(self, df, columns_to_encode, string):

        selected_columns = df[columns_to_encode]

        encode = pd.get_dummies(selected_columns, columns = columns_to_encode, prefix = string)

        encoded_df = pd.concat([df.drop(columns = columns_to_encode), encode], axis=1)

        return encoded_df

    def run_apriori(self, encoded_df, numb):

        support = apriori(encoded_df, min_support = numb, use_colnames = True)

        return support

    def generate_rules(self, support, rule, numb):

        association_rule = association_rules(support, metric = rule, min_threshold = numb)

        return association_rule

    def filter_rule(self, rule, impacted_variable):

        filtered_rules = rule[rule['consequents'] == {impacted_variable}]

        return filtered_rules

In [ ]:
merged_df = pd.merge(pd.merge(accident_df, casualty_df, on = 'accident_index'), vehicle_df, on = 'accident_index')
merged_df

In [ ]:
merged_apriori = accident_apriori(merged_df)

In [ ]:
# Dataframe of accident_severity, speed_limit & urban_or_rural_area

accident_apriori_df = merged_apriori.get_dataframe_of_selected_variables(['accident_severity', 'speed_limit',
                                                                          'urban_or_rural_area'])
accident_apriori_df

In [ ]:
# One hot encode each column in accident_apriori_df and merge the results in a dataframe

apriori_encoded_df = merged_apriori.one_hot_encode( accident_apriori_df, ['accident_severity', 'speed_limit',
                                                    'urban_or_rural_area'], ('severity', 'speed', 'urban_or_rural'))

apriori_encoded_df

In [ ]:
# Running apriori on apriori_encoded_df to get support

freq_item_sets = merged_apriori.run_apriori(apriori_encoded_df, 0.2)
freq_item_sets

In [ ]:
# Generating association the lift rule on freq_item_sets

lift_rule = merged_apriori.generate_rules(freq_item_sets, 'lift', 0.5)
lift_rule

In [ ]:
# Filter association rules relevant to accident_severity

merged_apriori.filter_rule(lift_rule, 'severity_3')

## <h4 style = 'text-align : center'>5. Identify accidents in our region: Kingston upon Hull, Humberside, and the East Riding of Yorkshire etc.. Run clustering on this data</h4>

<h5 style = 'text-align : center'>Geographic clustering </h5>

In [ ]:
# Check catalogue_df for police_force
# Police_force 16 is Humberside which also covers Kingston upon Hull, East Riding of Yorkshire, North Lincolnshire &
# North East Lincolnshire

catalogue_df[(catalogue_df['table'] == 'accident') | (catalogue_df['field name'] == 'police_force')]

### I created a class geograhic_clustering that does the following:
+ Does Kmeans and Kmedoid clustering of humberside region

In [ ]:
class geographic_clustering:
    def __init__(self, dataframe, longitude, latitude):

        self.dataframe = dataframe
        self.longitude = longitude
        self.latitude = latitude

    def elbow_method(self, type_of_cluster):
            # Create an empty list to store inertia values for different k values

            inertia_values = []

            # Try different values of k (number of clusters)

            k_values = range(1, 10)

            for k in k_values:

                if type_of_cluster == 'KMeans':

                    # Create a KMeans object with the current k value

                    cluster_model = KMeans(n_clusters = k)

                else:
                    # Create a KMedoids object with the current k value

                    cluster_model = KMedoids(n_clusters = k)

                # Fit the clustering model to the data
                cluster_model.fit(self.dataframe)

                # Append the inertia (sum of squared distances) to the list
                inertia_values.append(cluster_model.inertia_)

            # Plot the inertia values against different k values
            plt.plot(k_values, inertia_values, marker='o')
            plt.xlabel('Number of Clusters (k)')
            plt.ylabel('Inertia Values')
            plt.title(f'Elbow method for {type_of_cluster} to determine optimal k')
            plt.xticks(k_values)

            return plt.show()


    def  kmeans_clustering(self, k):

        # Create a KMeans object with the number of clusters = k
        kmeans = KMeans(n_clusters = k)

        # Fit the K-means model to the data
        kmeans.fit(self.dataframe)

        # Get the cluster labels
        labels = kmeans.predict(self.dataframe)

        # Get the cluster centers
        centroids = kmeans.cluster_centers_

        # Get the inertia (sum of squared distances)

        inertia = kmeans.inertia_

        return (labels, centroids, inertia)


    def  kmedoids_clustering(self, k):

        # Create a kmedoids object with the number of clusters = k
        kmedoids = KMedoids(n_clusters = k)

        # Fit the kmedoids model to the data
        kmedoids.fit(self.dataframe)

        # Get the cluster labels
        labels = kmedoids.predict(self.dataframe)

        # Get the cluster centers
        centroids = kmedoids.cluster_centers_

        # Get the inertia (sum of squared distances)

        inertia = kmedoids.inertia_

        return (labels, centroids, inertia)


    def plot_clustering(self, labels, centroids, type_of_cluster):

        # Determine which data is x & y
        plt.scatter(self.dataframe.longitude, self.dataframe.latitude, c = labels,  cmap = 'viridis', s = 5)

        # Add labels to the plot and a title
        plt.scatter(centroids[:, 0], centroids[:, 1], color = 'red')

        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        plt.title(f'{type_of_cluster} clustering of Humberside Region')

        return plt.show()



    def plot_clustering_map(self, k, labels):

        colors = ['red', "green", "blue", "purple"]

        cluster_areas = []

        cluster_colors = []

        for i in range(k):

            cluster_areas.append(list(self.dataframe[[self.latitude, self.longitude]][labels == i].values))

            cluster_colors.append(f"<svg width='12' height='12' xmlns='http://www.w3.org/2000/svg'> \
                                    <circle cx='6' cy='6' r='3' fill={colors[i]} /></svg>")

        map = folium.Map(location = [53.768363, -0.599618], zoom_start = 9.45, tiles = "openstreetmap")


        for i, area in enumerate(cluster_areas):

             for location in area:

                folium.Marker(location, icon=folium.DivIcon(html = cluster_colors[i])).add_to(map)

        return map

In [ ]:
# Dataframe for longitude and latitude of Humberside

humberside_geographical_df = accident_df[accident_df['police_force'] == 16][['longitude', 'latitude']]
humberside_geographical_df

In [ ]:
geo_coord = geographic_clustering(humberside_geographical_df, 'longitude', 'latitude')

In [ ]:
geo_coord.elbow_method('KMeans')

In [ ]:
kmeans_clustering = geo_coord.kmeans_clustering(4)
kmeans_clustering[2]

In [ ]:
geo_coord.plot_clustering(kmeans_clustering[0], kmeans_clustering[1], 'KMeans')

In [ ]:
geo_coord.plot_clustering_map(4, kmeans_clustering[0])

In [ ]:
geo_coord.elbow_method('KMedoid')

In [ ]:
kmedoids_clustering = geo_coord.kmedoids_clustering(4)
kmedoids_clustering[2]

In [ ]:
geo_coord.plot_clustering(kmedoids_clustering[0], kmedoids_clustering[1], 'KMedoids')

In [ ]:
geo_coord.plot_clustering_map(4, kmedoids_clustering[0])

<h4 style = 'text-align : center'>6. Using outlier detection methods, identify unusual entries in your data set</h4>

### I created class that does the following:
+ Detects outlier methods with IQR
+ Detects outlier methods with Grubbs test
+ Detects outlier methods with Isolation forest
+ Plots a distribution to show multivariate outliers
+ Plots a map to show multivariate outliers

In [ ]:
class outlier_detection_methods:
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def IQR_method(self, column, numb):


        x = np.array(self.dataframe[column])

        q75, q25 = np.percentile(x, [75, 25])
        iqr = q75 - q25

        upper_quartile = (q75 + numb * iqr)

        lower_quartile = (q25 - numb * iqr)

        outliers = self.dataframe[(self.dataframe[column] < lower_quartile) |
                                  (self.dataframe[column] > upper_quartile)][column].unique()

        return outliers


    def get_critical_g(self, size, alpha):
        tdist = stats.t.ppf(1 - alpha / (2 * size), size - 2)

        numerator = (size - 1) * np.sqrt(np.square(tdist))
        denominator = np.sqrt(size) * np.sqrt(size - 2 + np.square(tdist))

        critical_value = numerator / denominator

        return critical_value


    def grubbs_test(self, column, alpha):

        x = np.array(self.dataframe[column])

        outliers = []

        while True:
            avg = np.mean(x)
            absvalues = abs(x - avg)
            maxvalue = max(absvalues)
            value = x[list(absvalues).index(maxvalue)]
            stdev = np.std(x)
            G  = maxvalue / stdev

            critical_g = self.get_critical_g(len(x), alpha)

            if G > critical_g:

                x = x[x != value]

                outliers.append(value)

            else:

                break

        return np.array(outliers)

    def isolation_forest(self, columns, numb):

        iforest = IsolationForest(contamination = numb)

        outliers = iforest.fit_predict(self.dataframe[columns])

        df_outliers = self.dataframe[outliers == -1]

        df_non_outliers = self.dataframe[outliers == 1]

        return (df_outliers, df_non_outliers)


    def plot_multivariate_outliers(self, column_1, column_2, numb):

        df_outliers, df_non_outliers = self.isolation_forest([column_1, column_2], numb)

        sns.scatterplot(data = df_outliers, x = column_1, y =  column_2, s = 5, color = 'violet')

        sns.scatterplot(data = df_non_outliers, x = column_1, y =  column_2, s = 5, color = 'pink')

        plt.title(f'Distribution of {column_1} and {column_2} multivariate outliers')

        return plt.show()

    def plot_outlier_location_map(self, column_1, column_2, numb):

        df_outliers = self.isolation_forest([column_1, column_2], numb)[0]

        map = folium.Map(location = [56, -1], zoom_start = 5.4999999, titles = 'openstreetmap')

        locations_of_outliers = df_outliers[[column_2, column_1]].values

        color = """<svg width="12" height="12" xmlns="http://www.w3.org/2000/svg">
                   <circle cx="6" cy="6" r="2" fill="violet" /></svg>"""

        for location in locations_of_outliers:

            folium.Marker(location, icon = folium.DivIcon(html = color)).add_to(map)

        return map



In [ ]:
outlier_df = merged_df[['location_easting_osgr', 'location_northing_osgr', 'longitude', 'latitude', 'number_of_vehicles',
                        'number_of_casualties','decimal_time', 'age_of_casualty', 'age_of_driver', 'engine_capacity_cc',
                        'age_of_vehicle']]
outlier_df.head()

In [ ]:
outlier_variables = outlier_detection_methods(outlier_df)

<h3 style = 'text-align : center'>Univariate outliers</h3>

<h4 style = 'text-align : center'> number_of_vehicles</h4>

In [ ]:
outlier_variables.IQR_method('number_of_vehicles', 1.5)

In [ ]:
outlier_variables.grubbs_test('number_of_vehicles', 0.05)

<h4 style = 'text-align : center'>number_of_casualties</h4>

In [ ]:
outlier_variables.IQR_method('number_of_casualties', 1.5)

In [ ]:
outlier_variables.grubbs_test('number_of_casualties', 0.05)

<h4 style = 'text-align : center'>decimal_time</h4>

In [ ]:
outlier_variables.IQR_method('decimal_time', 1.5)

In [ ]:
outlier_variables.grubbs_test('decimal_time', 0.05)

<h4 style = 'text-align : center'>age_of_casualty</h4>

In [ ]:
outlier_variables.IQR_method('age_of_casualty', 1.5)

In [ ]:
outlier_variables.grubbs_test('age_of_casualty', 0.05)

<h4 style = 'text-align : center'>age_of_driver</h4>

In [ ]:
outlier_variables.IQR_method('age_of_driver', 1.5)

In [ ]:
outlier_variables.grubbs_test('age_of_driver', 0.05)

<h4 style = 'text-align : center'>engine_capacity_cc</h4>

In [ ]:
outlier_variables.IQR_method('engine_capacity_cc', 1.5)

In [ ]:
outlier_variables.grubbs_test('engine_capacity_cc', 0.05)

<h4 style = 'text-align : center'>age_of_vehicle</h4>

In [ ]:
outlier_variables.IQR_method('age_of_vehicle', 1.5)

In [ ]:
outlier_variables.grubbs_test('age_of_vehicle', 0.05)

<h3 style = 'text-align : center'>Multivariate outliers</h3>

In [ ]:
outlier_variables.isolation_forest(['longitude','latitude'], 0.01)[0]

In [ ]:
outlier_variables.plot_multivariate_outliers('longitude','latitude', 0.01)

In [ ]:
outlier_variables.plot_outlier_location_map('longitude', 'latitude', 0.01)

In [ ]:
outlier_variables.isolation_forest(['location_easting_osgr','location_northing_osgr'], 0.01)[0]

In [ ]:
outlier_variables.plot_multivariate_outliers('location_easting_osgr','location_northing_osgr', 0.01)

<h4 style = 'text-align : center'>7. Can you develop a classification model using the provided data that accurately predicts fatal injuries sustained in road traffic accidents, with the aim of informing and improving road safety measures</h4>

In [ ]:
# Check catalogue_df for accident_severity

catalogue_df[(catalogue_df['table'] == 'accident') | (catalogue_df['field name'] == 'accident_severity')]

In [ ]:
# Created a series which has True for fatal accidents and False for non fatal accidents

fatal_accidents = (merged_df['accident_severity'] == 1)
fatal_accidents

In [ ]:
fatal_accidents.value_counts()

Feature selection

In [ ]:
# Feature selection

columns_to_drop = ['accident_index','accident_year_x','accident_reference', 'accident_severity',
                   'did_police_officer_attend_scene_of_accident', 'date', 'converted_time', 'vehicle_index',
                   'casualty_index', 'casualty_severity', 'casualty_reference', 'lsoa_of_accident_location',
                   'local_authority_district', 'local_authority_highway', 'generic_make_model', 'vehicle_reference_x']

In [ ]:
merged_df = merged_df.drop(columns = columns_to_drop)

# Keep columns that are numbers in the data frame.
merged_df = merged_df.select_dtypes([np.number])

merged_df.head()

### I created a class that does the following
+ Balances a dataframe and series
+ Get the best sixteen variables to train my model
+ Builds four models  - Decision tree, random forest, XGBoost and Stacker
+ Prints the classification reports for all the models
+ Prints the Confusion matrix for all the models
+ Plots the boxplot for all the models

In [ ]:
class fatal_injury_classification:
    def __init__(self, dataframe, series):

        self.dataframe = dataframe

        self.series = series

        self.balanced_df, self.balanced_series = self.balance_dataframe_and_series()

        self.X_train, self.X_test, self.y_train, self.y_test = self.get_test_and_training_samples()

        self.models = self.get_models()

        self.y_pred_list = self.get_predictions()

    def balance_dataframe_and_series(self):

        # Creating a balanced the dataset using RandomUnderSampler

        self.balanced_df, self.balanced_series = RandomUnderSampler().fit_resample(self.dataframe, self.series)

        return (self.balanced_df, self.balanced_series)


    def get_kbest(self):
        selector = SelectKBest(f_classif, k = 16)
        selector.fit(self.balanced_df, self.balanced_series)
        mask = selector.get_support()
        names = selector.feature_names_in_[mask]
        scores = selector.scores_[mask]

        plt.bar(names, scores)
        plt.xticks(rotation = 'vertical')

        return plt.show()

    def get_test_and_training_samples(self):

        selector = SelectKBest(f_classif, k = 5)
        selector.fit(self.balanced_df, self.balanced_series)
        mask = selector.get_support()
        names = selector.feature_names_in_[mask]

        X = self.balanced_df[names] # (features)
        y = self.balanced_series # (labels)

        # Split the data into training and test sets (80% train, 20% test)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

        return (X_train, X_test, y_train, y_test)


    def get_models(self):

        models = dict()

        models['Decision_Tree'] = DecisionTreeClassifier()
        models['Random_Forest'] = RandomForestClassifier()
        models['XGBoost'] = XGBClassifier()
        models['Stacker'] = self.get_stacking()

        return models

    def get_stacking(self):

        level0 = []
        level0.append(('Decision_Tree', DecisionTreeClassifier()))
        level0.append(('Random_Forest', RandomForestClassifier()))
        level0.append(('XGBoost', XGBClassifier()))


        level1 = LogisticRegression()

        model = StackingClassifier(estimators = level0, final_estimator = level1, cv = 5)

        return model

    def get_predictions(self):

        y_pred_list = []

        for model_name, model in self.models.items():

            model.fit(self.X_train, self.y_train)

            y_pred = model.predict(self.X_test)

            y_pred_list.append(y_pred)

        return y_pred_list


    def cross_validation(self, model):

        models_list = list(self.models.keys())

        cv_scores = {}

        for model_name, model in self.models.items():

            cv_scores[model_name] = cross_validate(

                model, self.X_train, self.y_train,

                scoring = {

                    'accuracy': make_scorer(accuracy_score),
                    'precision': make_scorer(precision_score),
                    'recall': make_scorer(recall_score),
                    'f1': make_scorer(f1_score)
                },

                cv = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 3, random_state = 1),

                error_score='raise'
            )

        return cv_scores


    def get_classification_report(self, y_pred_list):

        models_list = list(self.models.keys())

        for i, model_name in enumerate(models_list):

            print(f'{model_name} Classification Report')
            print(classification_report(self.y_test, y_pred_list[i]), '\n' * 5)


    def plot_confusion_matrix(self, y_pred_list):

        models_list = list(self.models.keys())

        for i, model_name in enumerate(models_list):

            model = self.models[model_name]

            model.fit(self.X_train, self.y_train)

            # Predict classes for the test set
            y_pred = model.predict(self.X_test)

            # Generate and display the confusion matrix
            disp = ConfusionMatrixDisplay.from_estimator(model, self.X_test, self.y_test, display_labels=['Non Fatal', 'Fatal'])
            disp.figure_.suptitle(f'{model_name} Confusion Matrix')

            plt.show()

    def plot_cross_validation_results(self, cv_scores):

        models_list = list(self.models.keys())

        accuracy_scores = [cv_scores[model_name]['test_accuracy'] for model_name in models_list]

        plt.figure(figsize=(10, 6))
        plt.boxplot(accuracy_scores, labels = models_list)
        plt.title('Cross Validation Accuracy Results')
        plt.ylabel('Accuracy Score')
        plt.xticks(rotation = 'vertical')

        plt.show()


In [ ]:
# Gets an instance of the class fatal injury classification

classification = fatal_injury_classification(merged_df, fatal_accidents)

In [ ]:
# Gets Kbest

classification.get_kbest()

In [ ]:
# Get X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = classification.get_test_and_training_samples()

In [ ]:
# Get the models

models = classification.get_models()

In [ ]:
# Get y_pred_list

y_pred_list = classification.get_predictions()

In [ ]:
scores = classification.cross_validation(models)

In [ ]:
# Method to print the classification report of all the models

classification.get_classification_report(y_pred_list)

In [ ]:
# Method to print the confusion matrix of all the models

classification.plot_confusion_matrix(y_pred_list)

In [ ]:
# PLot of cross validation accuracy scores

classification.plot_cross_validation_results(scores)